In [1]:
import pandas as pd
import seaborn as sns
import folium
from folium import plugins
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize


file_path = 'C:\\Users\\isabe\\Desktop\\ANALISE_MEC\\'

prouni_2016 = pd.read_csv(file_path + 'pda-prouni-2016.csv', sep=';', encoding='ISO-8859-1')

municipios_brasil = pd.read_csv(file_path + 'MunicipiosBrasil.csv', usecols=['MUNICIPIO', 'UF',
                                                                             'LATITUDE', 'LONGITUDE'],
                                sep=';', encoding='ISO-8859-1')

prouni_2016 = prouni_2016.fillna('')
municipios_brasil = municipios_brasil.fillna('')

municipios_brasil['LATITUDE'] = municipios_brasil['LATITUDE'].apply(lambda x: (x.replace(',','.')).strip())
municipios_brasil['LATITUDE'] = municipios_brasil['LATITUDE'].astype(float)

municipios_brasil['LONGITUDE'] = municipios_brasil['LONGITUDE'].apply(lambda x: (x.replace(',','.')).strip())
municipios_brasil['LONGITUDE'] = municipios_brasil['LONGITUDE'].astype(float)

def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

prouni_2016['MUNICIPIO_BENEFICIARIO_BOLSA'] = prouni_2016['MUNICIPIO_BENEFICIARIO_BOLSA'].apply(lambda x: remover_acentos(x))

df_gerar_mapa = pd.merge(prouni_2016.drop_duplicates(), municipios_brasil.drop_duplicates(), left_on=['MUNICIPIO_BENEFICIARIO_BOLSA', 'SIGLA_UF_BENEFICIARIO_BOLSA'], 
                 right_on=['MUNICIPIO', 'UF'], how='left')

df_gerar_mapa = df_gerar_mapa[(~df_gerar_mapa['LATITUDE'].isnull()) | (~df_gerar_mapa['LONGITUDE'].isnull())]

df_municipio_bolsa = df_gerar_mapa.assign(QUANTIDADE_OCORRENCIAS_CIDADE=df_gerar_mapa.groupby('MUNICIPIO_BENEFICIARIO_BOLSA').MUNICIPIO_BENEFICIARIO_BOLSA.transform('count'))
df_municipio_bolsa = df_municipio_bolsa.sort_values(['QUANTIDADE_OCORRENCIAS_CIDADE','MUNICIPIO_BENEFICIARIO_BOLSA'],ascending=[True,False])

coordenadas = []
lat = df_municipio_bolsa['LATITUDE'][:500].values
long = df_municipio_bolsa['LONGITUDE'][:500].values

mapa = folium.Map(location=[-15.788497,-47.879873],tiles='Stamen Toner',zoom_start=4)

for la,lo in zip(lat,long):
    coordenadas.append([la,lo])

mapa.add_child(plugins.HeatMap(coordenadas))
mapa

# O mapa a seguir mostra quais foram as cidades onde obtiveram bolsas no programa prouni, as áreas com maior concentração
#são representadas de forma mais "quente" e as com menos quantidade "esfria". Lembrando que não são os dados totais, e sim
#uma pequena amostra (500 linhas de valores). Para coloração do mapa, foi utilizada um arquivo com extensão .csv retirado
#de um site do governo, onde contém todos os municípios brasileiros e sua latitude e longitude, que foram necessárias para
#composição do mapa

In [2]:
df_cursos_bolsa = prouni_2016.assign(QUANTIDADE_OCORRENCIAS_CURSO=prouni_2016.groupby('NOME_CURSO_BOLSA').NOME_CURSO_BOLSA.transform('count'))
df_cursos_bolsa = df_cursos_bolsa.sort_values(['QUANTIDADE_OCORRENCIAS_CURSO','NOME_CURSO_BOLSA'],ascending=[False,True])
df_cursos_bolsa = df_cursos_bolsa.drop_duplicates('NOME_CURSO_BOLSA')
df_cursos_bolsa[:5]

# Aqui retornamos quais são os cursos que mais obtiveram bolsas do programa prouni do ano de 2016 e na coluna "QUANTIDADE_
#OCORRENCIAS_CURSO", está a quantidade de ocorrências por ordem crescente desses 5 principais cursos.

,ANO_CONCESSAO_BOLSA,CODIGO_EMEC_IES_BOLSA,NOME_IES_BOLSA,TIPO_BOLSA,MODALIDADE_ENSINO_BOLSA,NOME_CURSO_BOLSA,NOME_TURNO_CURSO_BOLSA,CPF_BENEFICIARIO_BOLSA,SEXO_BENEFICIARIO_BOLSA,RACA_BENEFICIARIO_BOLSA,DT_NASCIMENTO_BENEFICIARIO,BENEFICIARIO_DEFICIENTE_FISICO,REGIAO_BENEFICIARIO_BOLSA,SIGLA_UF_BENEFICIARIO_BOLSA,MUNICIPIO_BENEFICIARIO_BOLSA,QUANTIDADE_OCORRENCIAS_CURSO
0,2016,20,UNIVERSIDADE DE PASSO FUNDO,BOLSA PARCIAL 50%,PRESENCIAL,Administração,Noturno,***847010**,Feminino,Branca,06-12-1995,NÃO,SUL,RS,SOLEDADE,24567
8,2016,423,UNIVERSIDADE REGIONAL INTEGRADA DO ALTO URUGUA...,BOLSA INTEGRAL,PRESENCIAL,Direito,Noturno,***110480**,Feminino,Branca,13-01-1998,NÃO,SUL,RS,ERECHIM,18731
94,2016,423,UNIVERSIDADE REGIONAL INTEGRADA DO ALTO URUGUA...,BOLSA INTEGRAL,PRESENCIAL,Pedagogia,Noturno,***968730**,Feminino,Branca,06-11-1987,NÃO,SUL,RS,SANTO ANGELO,16416
4,2016,423,UNIVERSIDADE REGIONAL INTEGRADA DO ALTO URUGUA...,BOLSA PARCIAL 50%,PRESENCIAL,Ciências Contábeis,Noturno,***560300**,Masculino,Branca,07-05-1994,NÃO,SUL,RS,ERECHIM,12994
32,2016,423,UNIVERSIDADE REGIONAL INTEGRADA DO ALTO URUGUA...,BOLSA INTEGRAL,PRESENCIAL,Engenharia Civil,Noturno,***143550**,Masculino,Branca,18-03-1984,NÃO,SUL,RS,SANTO ANGELO,10658
